In [1]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import plotly.offline as pyo
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

pd.set_option("display.max_columns", None)

def visualizza_grafico(ticker):
    import plotly.graph_objs as go
    fig = fx.grafico_base(ticker)
    tg = go.Scatter(
        x = ticker[ticker['Target'] == True].index,
        y = ticker[ticker['Target'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 15,
            color = 'rgba(0, 250, 0, .9)'
        ),
        name = 'target'
    )
    fig.add_trace(tg, row=1, col=1)
    pyo.plot(fig, filename="prova.html", auto_open=True)

def converti_in_XY(features, target):
    look_back = 5
    X, Y = [], []
    for i in range(look_back, len(features)):
        X.append(features.iloc[i - look_back:i].values)
        Y.append(target[i])
    X, Y = np.array(X), np.array(Y)
    return X, Y    

def addestramento(model, features, target):    
    X, Y = converti_in_XY(features, target)
    Y = np.where(Y, 1, 0)
    class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(class_weights) 

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    if model == 0:
        model = keras.Sequential([
            layers.LSTM(200, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 1
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 2
            layers.Dropout(0.2),
            layers.LSTM(200),  # Ultimo layer LSTM
            layers.Dropout(0.2),
            layers.Dense(1, activation='sigmoid')
        ])
    custom_optimizer = Adam(learning_rate=0.0001)

    model.compile(
        optimizer=custom_optimizer,
        loss='binary_crossentropy',
        metrics=[
            Precision(name='precision'),
            Recall(name='recall'),
            AUC(name='auc'),
            'accuracy'
        ]
    )

    scaler = StandardScaler()
    n_samples, n_timesteps, n_features = X.shape
    X_reshape = X.reshape((n_samples, n_timesteps * n_features))
    scaler.fit_transform(X_reshape)
    X_train, X_test, Y_train, Y_test = train_test_split(X_reshape, Y, test_size=0.2, random_state=42)
    X_train_scaled = X_train.reshape((-1, n_timesteps, n_features))
    X_test_scaled = X_test.reshape((-1, n_timesteps, n_features))
    
    model.fit(
        X_train_scaled, Y_train,
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        validation_data=(X_test_scaled, Y_test),
        callbacks=[
            early_stopping
        ]
    )

    loss, precision, recall, auc, accuracy = model.evaluate(X_test_scaled, Y_test)
    print(f"\033[42mLoss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\033[0m")
    with open("result.txt", "a") as f:
        f.write(f"Loss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\n")
    return model, scaler



In [2]:
elenco_usin = pd.read_excel("DeGiro con formule.xlsx")
elenco_usin = elenco_usin.loc[elenco_usin["Categoria"] != "D", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
tot_usin = len(elenco_usin)
i = 1
model = 0
for index, usin in elenco_usin.iterrows():       
    print(f"\r{i} di {tot_usin}) Caricamento ticker {usin['Ticker']}                           ", end=' ', flush=True)
    ticker = yf.download(usin["USIN"], start='2005-01-01', end='2023-07-31', progress=False)
    if len(ticker) < 500:
        continue
    else:
        ticker = fx.crea_indicatori(ticker)
        features = ticker.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Max_High_Futuro_20d", "Min_Low_Futuro_20d",
                                        "Drawdown_20d", "Perc_Max_High_Futuro_20d", "Perc_Drawdown_20d", "MaxMinRel", "guadagno_max_perc"
                                       ])        
        target = (ticker["Perc_Max_High_Futuro_20d"] >= 20) & (ticker["Perc_Drawdown_20d"] <=5)
        ticker["Target"] = target
        model, scaler = addestramento(model, features=features, target=target)        
        i += 1
    
os.remove("C:/Users/932197/Documents/dati_locali/last_ticker.txt")

print("fine addestramento")

12 di 2581) Caricamento ticker ABCL                            [0.5826087  3.52631579]
Epoch 1/100
14/14 [==============================] - 14s 254ms/step - loss: 0.7104 - precision: 0.1568 - recall: 0.4531 - auc: 0.4778 - accuracy: 0.5537 - val_loss: 0.6909 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_accuracy: 0.8889
Epoch 2/100
14/14 [==============================] - 0s 27ms/step - loss: 0.7100 - precision: 0.1429 - recall: 0.7031 - auc: 0.4783 - accuracy: 0.3248 - val_loss: 0.7124 - val_precision: 0.1111 - val_recall: 1.0000 - val_auc: 0.5000 - val_accuracy: 0.1111
Epoch 3/100
14/14 [==============================] - 0s 23ms/step - loss: 0.7097 - precision: 0.1386 - recall: 0.7188 - auc: 0.4713 - accuracy: 0.2897 - val_loss: 0.7079 - val_precision: 0.1111 - val_recall: 1.0000 - val_auc: 0.5000 - val_accuracy: 0.1111
Epoch 4/100
14/14 [==============================] - 0s 25ms/step - loss: 0.7106 - precision: 0.1538 - recall: 0.8438 - auc: 0.4637 - a

15/15 [==============================] - 0s 26ms/step - loss: 0.7102 - precision: 0.1733 - recall: 0.8143 - auc: 0.5770 - accuracy: 0.3910 - val_loss: 0.7050 - val_precision: 0.1026 - val_recall: 1.0000 - val_auc: 0.5000 - val_accuracy: 0.1026
Epoch 3/100
15/15 [==============================] - 0s 25ms/step - loss: 0.7150 - precision: 0.1527 - recall: 0.8571 - auc: 0.4353 - accuracy: 0.2671 - val_loss: 0.7189 - val_precision: 0.1026 - val_recall: 1.0000 - val_auc: 0.5000 - val_accuracy: 0.1026
Epoch 4/100
15/15 [==============================] - 0s 27ms/step - loss: 0.7113 - precision: 0.1532 - recall: 0.9714 - auc: 0.5121 - accuracy: 0.1923 - val_loss: 0.7142 - val_precision: 0.1026 - val_recall: 1.0000 - val_auc: 0.5000 - val_accuracy: 0.1026
Epoch 5/100
15/15 [==============================] - 0s 26ms/step - loss: 0.7135 - precision: 0.1441 - recall: 0.9143 - auc: 0.4722 - accuracy: 0.1752 - val_loss: 0.7215 - val_precision: 0.1026 - val_recall: 1.0000 - val_auc: 0.5000 - val_accur

114/114 [==============================] - 2s 19ms/step - loss: 0.6362 - precision: 0.1234 - recall: 0.4916 - auc: 0.6429 - accuracy: 0.6736 - val_loss: 0.6180 - val_precision: 0.2326 - val_recall: 0.4000 - val_auc: 0.6616 - val_accuracy: 0.8418
Epoch 22/100
114/114 [==============================] - 2s 21ms/step - loss: 0.6319 - precision: 0.1375 - recall: 0.4848 - auc: 0.6479 - accuracy: 0.7099 - val_loss: 0.8786 - val_precision: 0.0902 - val_recall: 0.8800 - val_auc: 0.5931 - val_accuracy: 0.2582
Epoch 23/100
114/114 [==============================] - 3s 22ms/step - loss: 0.6418 - precision: 0.1175 - recall: 0.5219 - auc: 0.6408 - accuracy: 0.6412 - val_loss: 0.6783 - val_precision: 0.0875 - val_recall: 0.8000 - val_auc: 0.6210 - val_accuracy: 0.2956
Epoch 24/100
114/114 [==============================] - 2s 19ms/step - loss: 0.6341 - precision: 0.1168 - recall: 0.5118 - auc: 0.6436 - accuracy: 0.6445 - val_loss: 0.6371 - val_precision: 0.2411 - val_recall: 0.3600 - val_auc: 0.6266 

114/114 [==============================] - 2s 21ms/step - loss: 0.6857 - precision: 0.0169 - recall: 0.5500 - auc: 0.4821 - accuracy: 0.4648 - val_loss: 0.6805 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_accuracy: 0.9813
Epoch 10/100
114/114 [==============================] - 2s 21ms/step - loss: 0.6877 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4827 - accuracy: 0.9456 - val_loss: 0.6638 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_accuracy: 0.9813
Epoch 11/100
114/114 [==============================] - 2s 21ms/step - loss: 0.6838 - precision: 0.0175 - recall: 0.7167 - auc: 0.5191 - accuracy: 0.3302 - val_loss: 0.6791 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_accuracy: 0.9813
Epoch 12/100
114/114 [==============================] - 3s 23ms/step - loss: 0.6855 - precision: 0.0146 - recall: 0.1167 - auc: 0.4895 - accuracy: 0.8560 - val_loss: 0.6759 - val_precision: 0.0000e+00 - v

KeyboardInterrupt: 

In [ ]:
def test():
    X, Y = converti_in_XY(features, target)
    n_samples, n_timesteps, n_features = X.shape
    X_reshape = X.reshape((n_samples, n_timesteps * n_features))
    scaler.transform(X_reshape)
    X_scaled = X_reshape.reshape((n_samples, n_timesteps, n_features))

    # Ottiene le previsioni per tutto il set X
    previsioni_totali = model.predict(X_scaled)

    # Trasforma in serie pandas, ignorando i primi 30 valori (look_back)
    previsioni_serie = pd.Series(previsioni_totali.flatten(), index=ticker.index[n_timesteps:])

    # Inserisce nel DataFrame originale
    ticker['Previsto'] = previsioni_serie
    ticker.to_excel("prova.xlsx")